<a href="https://colab.research.google.com/github/ettore9x9/Detectron2-ta/blob/main/DetectTraversable_Detectron2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%%capture
# Install Detectron2
!python -m pip install pyyaml==5.1
import sys, os, distutils.core
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))
!python -m pip install labelme

In [ ]:
# Import and print versions
import torch, detectron2
!nvcc --version
TORCH_VERSION = ".".join(torch.__version__.split(".")[:2])
CUDA_VERSION = torch.__version__.split("+")[-1]
print("torch: ", TORCH_VERSION, "; cuda: ", CUDA_VERSION)
print("detectron2:", detectron2.__version__)

In [ ]:
# Some basic setup:
# Setup detectron2 logger
from detectron2.utils.logger import setup_logger
setup_logger()

# import some common libraries
import numpy as np
import json, cv2, random
from google.colab.patches import cv2_imshow
import scipy.io as sio
import json
from skimage import measure

# import some common detectron2 utilities
from detectron2 import model_zoo
from detectron2.engine import DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import Visualizer
from detectron2.data import MetadataCatalog, DatasetCatalog

In [ ]:
# download datasets
os.makedirs("./zipData", exist_ok=True)
!wget -P ./zipData https://mikeprocopio.com/downloads/lagr/labeled_lagr_data_640x480_DS1A.ZIP
!wget -P ./zipData https://mikeprocopio.com/downloads/lagr/labeled_lagr_data_640x480_DS1B.ZIP
!wget -P ./zipData https://mikeprocopio.com/downloads/lagr/labeled_lagr_data_640x480_DS2A.ZIP
!wget -P ./zipData https://mikeprocopio.com/downloads/lagr/labeled_lagr_data_640x480_DS2B.ZIP
!wget -P ./zipData https://mikeprocopio.com/downloads/lagr/labeled_lagr_data_640x480_DS3A.ZIP
!wget -P ./zipData https://mikeprocopio.com/downloads/lagr/labeled_lagr_data_640x480_DS3B.ZIP

In [ ]:
import base64
from PIL import Image

def convert_mat_to_png(matfile, outputfile):
    mat_contents = sio.loadmat(matfile)
    img_rgb = mat_contents['im_rgb']
    out = Image.fromarray(img_rgb)
    out.save(outputfile)

def convert_mat_to_mask(matfile, outputfile):
    mat = sio.loadmat(matfile)
    mask = mat['manual_human_labeling_mask']
    out = Image.fromarray(mask.astype(np.uint8), mode='L')
    out.save(outputfile)

In [ ]:
# decompress data
import shutil
import zipfile

datasets_paths = ['/content/zipData/labeled_lagr_data_640x480_DS1A.ZIP',
                  '/content/zipData/labeled_lagr_data_640x480_DS1B.ZIP',
                  '/content/zipData/labeled_lagr_data_640x480_DS2A.ZIP',
                  '/content/zipData/labeled_lagr_data_640x480_DS2B.ZIP',
                  '/content/zipData/labeled_lagr_data_640x480_DS3A.ZIP',
                  '/content/zipData/labeled_lagr_data_640x480_DS3B.ZIP'
                  ]

extract_path = '/content/data'
train_path   = '/content/train'
val_path     = '/content/val'
test_path    = '/content/test'

os.makedirs(extract_path, exist_ok=True)
os.makedirs(train_path,   exist_ok=True)
os.makedirs(val_path,     exist_ok=True)
os.makedirs(test_path,    exist_ok=True)

train_rgb_path  = '/content/train/rgb'
train_mask_path = '/content/train/mask'
val_rgb_path    = '/content/val/rgb'
val_mask_path   = '/content/val/mask'
test_rgb_path   = '/content/test/rgb'
test_mask_path  = '/content/test/mask'

os.makedirs(train_rgb_path,  exist_ok=True)
os.makedirs(train_mask_path, exist_ok=True)
os.makedirs(val_rgb_path,    exist_ok=True)
os.makedirs(val_mask_path,   exist_ok=True)
os.makedirs(test_rgb_path,   exist_ok=True)
os.makedirs(test_mask_path,  exist_ok=True)

train_ratio = 0.8
val_ratio   = 0.1
i = 0

for zip_file_path in datasets_paths:
  with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
      zip_ref.extractall(extract_path)

  files = os.listdir(extract_path)
  random.shuffle(files)

  split_train_index = int(len(files) * train_ratio)
  split_val_index   = int(len(files) * val_ratio) + split_train_index
  train_files = files[:split_train_index]
  val_files   = files[split_train_index:split_val_index]
  test_files  = files[split_val_index:]

  for filename in train_files:
      src_path      = os.path.join(extract_path,    filename)
      dst_path_rgb  = os.path.join(train_rgb_path,  "frame"+str(i)+".png")
      dst_path_mask = os.path.join(train_mask_path, "frame"+str(i)+".png")
      i = i+1
      convert_mat_to_png(src_path, dst_path_rgb)
      convert_mat_to_mask(src_path, dst_path_mask)

  for filename in val_files:
      src_path      = os.path.join(extract_path,  filename)
      dst_path_rgb  = os.path.join(val_rgb_path,  "frame"+str(i)+".png")
      dst_path_mask = os.path.join(val_mask_path, "frame"+str(i)+".png")
      i = i+1
      convert_mat_to_png(src_path, dst_path_rgb)
      convert_mat_to_mask(src_path, dst_path_mask)

  for filename in test_files:
      src_path      = os.path.join(extract_path,   filename)
      dst_path_rgb  = os.path.join(test_rgb_path,  "frame"+str(i)+".png")
      dst_path_mask = os.path.join(test_mask_path, "frame"+str(i)+".png")
      i = i+1
      convert_mat_to_png(src_path, dst_path_rgb)
      convert_mat_to_mask(src_path, dst_path_mask)

In [ ]:
import re

def get_data_dicts(directory):
    dataset_dicts = []
    rgb_dir  = directory + "/rgb"
    mask_dir = directory + "/mask"
    for filename in [file for file in os.listdir(rgb_dir)]:
        rgb_file  = os.path.join(rgb_dir, filename)
        mask_file = os.path.join(mask_dir, filename)

        record = {}
        record["file_name"] = rgb_file
        record["height"] = 640
        record["width"] = 480
        record["image_id"] = re.findall(r'\d+', filename)[-1]
        record["sem_seg_file_name"] = mask_file

        dataset_dicts.append(record)
    return dataset_dicts

In [ ]:
data_path = '/content/'

for d in ["train", "val", "test"]:
    DatasetCatalog.register(
        "category_" + d,
        lambda d=d: get_data_dicts(data_path+d)
    )

stuff_classes = ["traversable", "obstacle"]

MetadataCatalog.get("category_train").set(stuff_classes=stuff_classes)
MetadataCatalog.get("category_train").set(ignore_label=2)
MetadataCatalog.get("category_val").set(stuff_classes=stuff_classes)
MetadataCatalog.get("category_val").set(ignore_label=2)

In [ ]:
from detectron2 import model_zoo
from detectron2.engine import DefaultTrainer, DefaultPredictor
from detectron2.config import get_cfg
from detectron2.utils.visualizer import ColorMode, Visualizer
import detectron2.data.transforms as T
from detectron2.data.dataset_mapper import DatasetMapper
from detectron2.data import build_detection_train_loader
from detectron2.evaluation import SemSegEvaluator

In [ ]:
transform_list = [
    T.RandomBrightness(0.8, 1.2),
    T.RandomContrast(0.8, 1.2),
    T.RandomSaturation(0.8, 1.2),
    T.RandomFlip(prob=0.5, horizontal=True, vertical=False),
]

class MyTrainer(DefaultTrainer):

    @classmethod
    def build_evaluator(cls, cfg, dataset_name, output_folder=None):
        if output_folder is None:
            output_folder = os.path.join(cfg.OUTPUT_DIR, "inference")
        
            return SemSegEvaluator(
                    dataset_name,
                    distributed=True,
                    output_dir=output_folder,
                    )

    @classmethod
    def build_train_loader(cls, cfg):
        mapper = DatasetMapper(cfg, is_train=True, augmentations=transform_list)
        return build_detection_train_loader(cfg, mapper=mapper)

In [ ]:
# Semantic segmentation settings
cfg = get_cfg()
cfg.merge_from_file(model_zoo.get_config_file("Base-RCNN-FPN.yaml"))

cfg.DATASETS.TRAIN = ("category_train",)
cfg.DATASETS.TEST = ("category_test",)
cfg.MODEL.META_ARCHITECTURE = "SemanticSegmentor"
cfg.MODEL.WEIGHTS = "detectron2://ImageNetPretrained/MSRA/R-50.pkl"
cfg.MODEL.RESNETS.DEPTH = 50
cfg.INPUT.MIN_SIZE_TRAIN = (480)
cfg.INPUT.MAX_SIZE_TRAIN = (640)
cfg.DATALOADER.NUM_WORKERS = 2
cfg.MODEL.ROI_HEADS.NUM_CLASSES = 2

cfg.SOLVER.LR_SCHEDULER_NAME = "WarmupMultiStepLR"
cfg.SOLVER.MAX_ITER = 1000
cfg.SOLVER.BASE_LR = 0.02
cfg.SOLVER.MOMENTUM = 0.9
cfg.SOLVER.WEIGHT_DECAY = 0.0001
cfg.SOLVER.GAMMA = 0.1
cfg.SOLVER.WARMUP_FACTOR = 1.0 / 1000
cfg.SOLVER.WARMUP_ITERS = 300
cfg.SOLVER.IMS_PER_BATCH = 16

In [ ]:
os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg) 
trainer.resume_or_load(resume=False)

In [ ]:
trainer.train()

In [ ]:
cfg.MODEL.WEIGHTS = os.path.join(cfg.OUTPUT_DIR, "model_final.pth")
cfg.MODEL.ROI_HEADS.SCORE_THRESH_TEST = 0.7
predictor = DefaultPredictor(cfg)

In [ ]:
eval_dataset = get_data_dicts(data_path+'val')

In [ ]:
# Show images with semantic segmentation
import matplotlib.pyplot as plt

for d in random.sample(eval_dataset, 3):
    img = cv2.imread(d["file_name"])
    outputs = predictor(img)
    v = Visualizer(img[:, :, ::-1],
                   scale=0.8,
                   metadata=MetadataCatalog.get("category_val"),
                   instance_mode=ColorMode.SEGMENTATION
    )
    v = v.draw_sem_seg((outputs["sem_seg"].argmax(dim=0)).to("cpu"))
    plt.figure(figsize=(14, 10))

    # Transpose the image for display pourposes
    plt.imshow(cv2.cvtColor(v.get_image()[:, :, ::-1].transpose(1, 0, 2), cv2.COLOR_BGR2RGB))
    plt.show()

In [ ]:
from google.colab import files
!zip -r /content/out.zip /content/output
files.download("/content/out.zip")